In [2]:
import pandas as pd
import numpy as np
import gensim
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import torch.nn.functional as F
from torchtext import data
from torch.utils.data import TensorDataset

In [11]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [8]:
train_df = pd.read_csv("/content/drive/MyDrive/CUAI_Winter/train.tsv", delimiter='\t')
valid_df = pd.read_csv("/content/drive/MyDrive/CUAI_Winter/valid.tsv", delimiter='\t')

In [ ]:
sentences_train = [train_df['sent'][i] for i in range(len(train_df['sent']))]
sentences_valid = [valid_df['sent'][i] for i in range(len(valid_df['sent']))]

sentences_train_len = [len(train_df['sent'][i]) for i in range(len(train_df['sent']))]
sentences_valid_len = [len(valid_df['sent'][i]) for i in range(len(valid_df['sent']))]

max_train_sentence_length = max(sentences_train_len)
max_valid_sentence_length = max(sentences_valid_len)

print(max_train_sentence_length)
print(max_valid_sentence_length)

In [10]:
labels_train = train_df['sentiment'].to_list()
labels_valid = valid_df['sentiment'].to_list()

In [12]:
def load_word2vec_from_kyubyong():
  kyubyong_word2vec = gensim.models.Word2Vec.load("/content/drive/MyDrive/CUAI_Winter/ko.bin")
  return kyubyong_word2vec

In [13]:
word2vec = load_word2vec_from_kyubyong()

In [14]:
# aihub 감성 말뭉치 데이터로 추가 학습
aihub_sentences = sentences_train
def train_word2vec_with_aihub(sentences, word2vec):
  sentences = sentences + [['']]
  word2vec.build_vocab(sentences, update=True)
  word2vec.train(sentences, total_examples=word2vec.corpus_count, epochs=word2vec.epochs)
  return 

In [15]:
train_word2vec_with_aihub(aihub_sentences, word2vec)

In [ ]:
def word_to_index(word):
  if word in word2vec:
    return word2vec.wv.vocab[word].index
  else:
    return 0

In [ ]:
def sentences_to_indicies(sentences):
  sentences_of_indicies = []
  for sentence in sentences:
    indicies = list(map(word_to_index, sentence))
    sentences_of_indicies.append(indices)
  return sentences_of_indicies

In [ ]:
def pad_sequences(sentences_train):
  sentence_indicies = []
  for i in range(len(sentences_train)):
    number_of_additional_indicies = max_train_sentence_length - len(sentences_train[i])
    additional_indicies = number_of_additional_vectors * [0]
    sentences_of_indicies.append(sentences_train[i] + additional_indicies))
  
  return sentences_of_indicies

In [23]:
print(sentences_train[0])

['아내', '출산', '되', '신', '나']


In [ ]:
x_train = sentences_of_indicies(sentences_train)
x_valid = sentences_of_indicies(sentences_train)

In [ ]:
# wordvector => tensor
def wordindicies_to_tensor():
  return map(torch.tensor, (x_train, labels_train, x_valid, labels_valid))

In [ ]:
x_train, y_train, x_valid, y_valid = wordindicies_to_tensor()

In [ ]:
train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

In [ ]:
# 학습에 필요한 변수들의 정의
learning_rate = 0.001 # 학습률 
epochs = 10 # 전체 학습 순회 횟수
batch_size = 64 # 훈련에 사용할 미니 데이터셋의 크기(데이터의 개수)
dropout = 0.2
n_classes = len(train_df['sentiment'].unique())
loss_func = nn.functional.cross_entropy

input_size = 300
hidden_size = 128
num_layers = 1

weights = torch.FloatTensor(word2vec.wv.vectors)

In [ ]:
# Dataloader 
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size * 2, shuffle=False)

In [ ]:
# 단순한 시퀀스 모델보다 복잡한 모델을 구성해야 할 때는, nn.Module의 하위 클래스를 선언하고,
# 입력 텐서를 받아 다른 모듈 및 autograd 연산을 사용하여 출력 텐서를 만드는 forward method를 정의 합니다.
# 입력 텐서(문장)의 길이는 모두 같아야 한다.

class Rnn_Sentiment_Classification(nn.Module):
  # input_size: wordvector의 크기 200
  # hidden_size: hidden cell의 크기 128, 256
  def __init__(self, input_size, hidden_size, num_layers, dropout, n_classes):
    super(Rnn_Sentiment_Classification, self).__init__()
    # 매개변수를 생성하고 멤버 변수로 지정합니다.
    self.num_layers = num_layers
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding.from_pretrained(weight, padding_idx=0)
    self.embedding.requires_grad = False
    self.dropout = nn.Dropout(dropout)

    self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)

    self.out = nn.Linear(self.hidden_size, n_classes)

  def forward(self, input_tensor):
    # 첫번째 히든 스테이트를 0벡터로 초기화
    input_tensor = self.embedding(input_tensor)
    h_0 = self.initHidden(batch_size=input_tensor.size(0))

    # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
    x, _ = self.gru(input_tensor, h_0)

    # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
    h_t = x[:,-1,:]

    self.dropout(h_t)
    # (배치 크기, 은닉 상태의 크기) => (배치 크기, 출력층의 크기)
    logit = self.out(h_t)

    return logit

  def initHidden(self, batch_size=1):
    weight = next(self.parameters()).data
    return weight.new(self.num_layers, batch_size, self.hidden_size).zero_()

In [ ]:
def get_model():
  model = Rnn_Sentiment_Classification(input_size, hidden_size, num_layers, dropout, n_classes).to(DEVICE)
  return model, optim.SGD(model.parameters(), lr=learning_rate)

def loss_batch(model, loss_func, x_batch, y_batch, opt=None):
  loss = loss_func(model(x_batch), y_batch)

  if opt is not None:
    loss.backward()
    opt.step()
    opt.zero_grad()

  return loss.item(), len(x_batch)

def fit(epochs, model, loss_func, opt, train_dataloader, valid_dataloader):
  for epoch in range(epochs):
    model.train()
    for x_batch, y_batch in train_dataloader:
      x_batch = wordbatch_to_indexbatch(x_batch)
      loss_batch(model, loss_func, x_batch, y_batch, opt)
    
    model.eval()
    with torch.no_grad():
      losses, nums = zip(*[loss_batch(model, loss_func, x_batch, y_batch) for x_batch, y_batch in valid_dataloader])
      val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

    print(epoch, val_loss)

def get_data(train_dataset, valid_dataset, batch_size):
  return (DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(valid_dataset, batch_size=batch_size * 2, shuffle=False))

In [ ]:
# 손실 함수를 정의합니다.
# optimizer를 정의합니다.

def train_rnn_model(batch_size, epochs, model):
  # 총 batch_size번의 학습 => 나중에 이 학습을 epochs만큼 반복할 것임.
  # 전체 과정은 이렇다.
  # 데이터의 미니배치를 선택
  # 모델을 이용하여 예측 수행
  # 손실 계산
  # loss.backward()를 이용하여 모델의 기울기 업데이트 
  
  # 순전파 단계: 특정 shape의 tensor를 모델에 전달하여 예측값 y를 계산한다.

  # 예측값과 실제 값을 손실함수로 전달하여 손실을 계산한다. 손실함수는 손실을 갖는 텐서를 반환한다.

  # 역전파 단계를 실행하기 전에 변화도(gradient)를 0으로 초기화한다.

  # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를 계산한다.
 
  # 경사 하강법을 사용하여 가중치를 갱신한다.
  # with torch.no_grad():
  #   for param in model.parameters():
  #     param -= learning_rate * param.grad
  # 위 3 단계를 optimizer를 정의하여 최적화한다.
  
  # 최종 최적화 프로세스
  train_dataloader, valid_dataloader = get_data(train_dataset, valid_dataset)
  model, optimizer = get_model()
  fit(epochs, model, loss_func, optimizer, train_dataloader, valid_dataloader)
